# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Introduction</p>



This Notebook is first of series, the final target is build a automatic Vehicle number plate Detection. On this notebook I am building the Train Pipeline, 
 

<img src = "https://miro.medium.com/v2/resize:fit:1000/1*gg6B3sy57dXoV37_57hWHg.png" 
  style='  display: block;
          margin-left: auto;
          margin-right: auto;
         '> 

<p style="font-family:newtimeroman;font-size:90%;text-align:center;border-radius:10px 10px;">Source: https://javascript.plainenglish.io/building-a-robust-license-plate-detection-and-recognition-system-using-javascript-8c64b314eec2 </p>    
    

    


### This kernel is inpired and uses many resources:

**Thanks to my colleagues for their beautiful kernels**

 - [YOLOv8 Finetuning for PPE detection](https://www.kaggle.com/code/hinepo/yolov8-finetuning-for-ppe-detection/notebook) by @[hinepeo](https://www.kaggle.com/hinepo/code)
 
 - [Yolo V8 - Vehicles Detecting \ Counting](https://www.kaggle.com/code/paulojunqueira/yolo-v8-vehicles-detecting-counting/notebook) by @[paulojunqueira](https://www.kaggle.com/paulojunqueira)
 
**Other Good Resources**


- [Realtime Plate Reading (video) | YOLOv5 & easyOCR](https://www.kaggle.com/code/saworz/realtime-plate-reading-video-yolov5-easyocr) 

- [License Plate Detection | YOLOv5 + pyTesseRACt](https://www.kaggle.com/code/gowrishankarp/license-plate-detection-yolov5-pytesseract/notebook)





 
 


# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Summary</p>


<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">1 | Librarys and Packeges</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">2 | Configurations</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">3 | Utils</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">4 | Data</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">6 | YAML file</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">7 | Images</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">8 | Train Detection Model</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">9 | Results</p>
<p style="font-family:newtimeroman;font-weight:600;color:#682F2F;font-size:150%; border-bottom: 2px solid #682F2F;">10 | OCR</p>

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Librarys  and Packeges</p>

In [ ]:
! pip install --upgrade ultralytics -qq
!pip install GPUtil -qq

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import shutil
import re
import glob
import subprocess
import random
import yaml
import tqdm
import gc
 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

import IPython.display as display
from IPython.display import Video
from PIL import Image
import cv2

import ultralytics
from ultralytics import YOLO

import xml.etree.ElementTree as xet
from bs4 import BeautifulSoup

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import pytesseract

print('ultralytics version: ',ultralytics.__version__)

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Configuarations 🔧</p>

In [ ]:
class CFG:
    
    out_folder = f'/kaggle/working'
    class_name = ['car_plate']
    video_test_path = 'https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-'

    weights = 'yolov8s.pt'
    exp_name = 'car_plate_detection'
    img_size = (240,400)
    
    epochs = 50
    batch_size = 16

    optimizer = 'auto' # SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto
    lr = 1e-3
    lr_factor = 0.01 #lo*lr_f
    weight_decay = 5e-4
    dropout = 0.0
    patience = int(0.7*epochs)
    profile = False
    label_smoothing = 0.0 
    seed=42
    
    confidance = 0.5

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Utils 🔧</p>

In [ ]:
#https://www.kaggle.com/code/hinepo/yolov8-finetuning-for-ppe-detection/notebook

def get_bbox(file_path):
    
    info = xet.parse(file_path)
    root = info.getroot()
    member_object = root.find('object')
    
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)
    
    return  xmin, xmax, ymin, ymax

def parse_xml_tags(data):
    """Parse xml label file, return image file name, and its coordinates as a dictionary
    """
    tags = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    Bs_data = BeautifulSoup(data, "xml")
    d = dict()

    for t in tags:
        text = Bs_data.find(t).text
        if all(c.isdigit() for c in text):
            d[t] = int(text)
        else:
            d[t] = text
    return d

def get_bbox_2(file_path):
    
    with open(file_path, 'r') as f:
        text = f.read()
        bbox = [float(x) for x in text.split(' ')[1:]]
        x_center ,y_center, width, height = bbox
        
        xmin = x_center - width
        xmax = x_center + width
        
        ymin = y_center - height
        ymax = y_center + height       
    
        return  xmin, xmax, ymin, ymax
    

def convert_xml_txt_yolo(file_path,w_image,h_image):
    
    with open(file_path,  'r') as f:
        label = parse_xml_tags(f.read())
  

#     dx = image.shape[1]-CFG.img_size[0]
#     dy = image.shape[0]-CFG.img_size[1]
    
    xmin = int(label['xmin'])
    xmax = int(label['xmax']) 
    ymin = int(label['ymin'])
    ymax = int(label['ymax'])
     
#     xmin = int(label['xmin']  + dx)
#     xmax = int(label['xmax'] + dx) 
#     ymin = int(label['ymin'] + dy)
#     ymax = int(label['ymax'] + dy)
    
    x_center = float((xmin+((xmax-xmin)/2))/w_image)
    y_center = float((ymin+((ymax-ymin)/2))/h_image)
    
    width = float((xmax-xmin)/w_image)
    height = float((ymax-ymin)/h_image)
    
    str_out = f'0 {x_center} {y_center} {width} {height}'
    
    return str_out

def display_image(image, print_info = True, hide_axis = False):
    if isinstance(image, str):  # Check if it's a file path
        img = Image.open(image)
        fig = plt.figure(figsize = (15,15))
        plt.imshow(img)
    elif isinstance(image, np.ndarray):  # Check if it's a NumPy array
        image = image[..., ::-1]  # BGR to RGB
        img = Image.fromarray(image)
        plt.imshow(img)
    else:
        raise ValueError("Unsupported image format")

    if print_info:
        print('Type: ', type(img), '\n')
        print('Shape: ', np.array(img).shape, '\n')

    if hide_axis:
        plt.axis('off')

    plt.show()

def create_dir(path):
    
    if not os.path.exists(path):
        os.mkdir(path)

        
def free_gpu_cache() -> None:
    print("Initial GPU Usage")
    gpu_usage()


    torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()


    
    

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Data 📊</p>

Labels are in XML format, it's need convert to yolo format

In [ ]:
anoattions_path_xml = glob.glob('/kaggle/input/car-plate-detection/annotations/*.xml')
image_paths = glob.glob('/kaggle/input/car-plate-detection/images/*.png')

info = xet.parse(anoattions_path_xml[0])
xet.dump(info)

## Train Text split

In [ ]:
index = np.arange(len(anoattions_path_xml))
np.random.shuffle(index)

val_index = index[:50]
# test_index = index[50:100]
train_index = index[50:]
# val_index = np.random.choice(index, size=50, replace=False)

print('Train Size: ', len(train_index))
print('Valid Size: ', len(val_index))
# print('Test Size: ', len(test_index))

In [ ]:
#crete paths for yolo labels
create_dir(CFG.out_folder)
datasets = ['train','valid']
folders = ['images','labels']
for datset in datasets:
    path_1 = CFG.out_folder + f'/{datset}'
    create_dir(path_1)
    for folder in folders:
        path_2 = CFG.out_folder + f'/{datset}/{folder}'
    
        create_dir(path_2)
    
        print(path_2)

## Convert to Yolo format

In [ ]:
for i, img_path in enumerate(image_paths):
    image = cv2.imread(img_path)
    
    resize_image = cv2.resize(image,CFG.img_size)
    h_image,w_image,_ = image.shape
    
    label_path = img_path.replace('images','annotations').replace('.png','.xml')
    
    
    label_text = convert_xml_txt_yolo(label_path,w_image,h_image)
    
    
    
    text_file_name = img_path.split('/')[-1].replace('.png','.txt')
    img_file_name = img_path.split('/')[-1]
#     print(img_file_name)
    

    if i in val_index:
        dataset = 'valid'
    elif i in train_index:
        dataset = 'train'
    elif i in test_index:
        dataset = 'test'
        
    
    text_path = f'{CFG.out_folder}/' + dataset +'/'+'/labels/' + text_file_name
    new_img_path = f'{CFG.out_folder}/' + dataset +'/images/'+ img_file_name
    
    shutil.copy2(img_path,new_img_path)
    #cv2.imwrite(new_img_path, resize_image)
    
    
    text_file = open(text_path, "w")
    text_file.write(label_text)
    text_file.close()

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">YAML file 📊</p>

In [ ]:
dict_file = {
    'train': os.path.join(CFG.out_folder, 'train'),
    'val': os.path.join(CFG.out_folder, 'valid'),
    'nc': len(CFG.class_name),
    'names': CFG.class_name
    }

with open(os.path.join('./', 'data.yaml'), 'w+') as file:
    yaml.dump(dict_file, file)
    
with open('./data.yaml', 'r') as file:
    data_yaml = yaml.safe_load(file)

print(yaml.dump(data_yaml))

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Images 🔬</p>

In [ ]:
def plot_random_images_from_folder(folder_path, num_images=20, seed=CFG.seed):
    
    random.seed(seed)

    # Get a list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg', '.gif'))]



    # Ensure that we have at least num_images files to choose from
    if len(image_files) < num_images:
        raise ValueError("Not enough images in the folder")

    # Randomly select num_images image files
    selected_files = random.sample(image_files, num_images)

    # Create a subplot grid
    num_cols = 5
    num_rows = (num_images + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))

    for i, file_name in enumerate(selected_files):
        # Open and display the image using PIL
        #img = Image.open(os.path.join(folder_path, file_name))
        img = cv2.imread(os.path.join(folder_path, file_name))
#         resize_img = cv2.resize(img,CFG.img_size)
#         w_img, h_img, _ = img.shape
        
        
    
        
#         x_center ,y_center, width, height = get_bbox_2(file_path = os.path.join(folder_path.replace('images','labels'), 
#                                                                                 file_name.replace('.png','.txt')))
        
        
        xmin, xmax, ymin, ymax = get_bbox(file_path=os.path.join('/kaggle/input/car-plate-detection/annotations', 
                                                                 file_name.replace('.png','.xml')))

#         x_center *= w_img
#         y_center *= h_img
#         width *= w_img
#         height *= h_img

#         xmin = int(x_center - (width/2))
#         xmax = int(x_center + (width/2))
        
#         ymin = int(y_center - (height/2))
#         ymax = int(y_center + (height/2))


        
        start_point = (xmin, ymax)
        end_point = (xmax, ymin)
        
        img = cv2.rectangle(img, start_point, end_point, (255,0,0), 3)
        
        
        if num_rows == 1:
            ax = axes[i % num_cols]
        else:
            ax = axes[i // num_cols, i % num_cols]
        
        ax.imshow(img)
        ax.axis('off')
        # ax.set_title(file_name)

    # Remove empty subplots
    for i in range(num_images, num_rows * num_cols):
        if num_rows == 1:
            fig.delaxes(axes[i % num_cols])
        else:
            fig.delaxes(axes[i // num_cols, i % num_cols])

    plt.tight_layout()
    plt.show()

In [ ]:
plot_random_images_from_folder(folder_path= '/kaggle/input/car-plate-detection/images', 
                               num_images=20, 
                               seed=CFG.seed)

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Train Detection Model</p>

In [ ]:
! wandb disabled

In [ ]:
model = YOLO(CFG.weights)

In [ ]:
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    
    if num_gpus > 1:
        train_device, test_device = 0,1
      
    else:
        train_device, test_device = 0,0        

    
    model.to(f'cuda:{train_device}')

    # Get information about each GPU
    for i in range(num_gpus):
        gpu_properties = torch.cuda.get_device_properties(i)
        print(f"\nGPU {i}: {gpu_properties.name}")
        print(f"  Total Memory: {gpu_properties.total_memory / (1024**3):.2f} GB")
        print(f"  CUDA Version: {gpu_properties.major}.{gpu_properties.minor}")
        

else:
    print("CUDA is not available. You can only use CPU.")
    train_device, test_device = 'cpu', 'cpu'
    model.to(train_device)
    
print('\n')
gpu_usage()

In [ ]:
print('Model: ', CFG.weights)
print('Device: ' ,model.device)

In [ ]:
%%time

### train
model.train(
    data = os.path.join(CFG.out_folder, 'data.yaml'),

    task = 'detect',

    #imgsz = (img_properties['height'], img_properties['width']),

    epochs = CFG.epochs,
    batch = CFG.batch_size,
    optimizer = CFG.optimizer,
    lr0 = CFG.lr,
    lrf = CFG.lr_factor,
    weight_decay = CFG.weight_decay,
    dropout = CFG.dropout,
    patience = CFG.patience,
    label_smoothing = CFG.label_smoothing,
    imgsz = 640,#CFG.img_size,

    name = CFG.exp_name,
    seed = CFG.seed,
    profile = False,
    
    val = True,
    amp = False,   #mixed precision 
    exist_ok = False, #overwrite experiment
    resume = False,
    device = train_device,
    verbose = False,
    single_cls = False,
)

## Save Model

In [ ]:
model.export(
    format = 'openvino', # openvino, onnx, engine, tflite
    imgsz = 640,#CFG.img_size,
    half = False,#fp16
    int8 = False,
    simplify = False,
    nms = False,
)

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Train Results</p>

In [ ]:
res_path = CFG.out_folder + '/runs/detect/' + os.listdir(CFG.out_folder + '/runs/detect/')[-1]
!tree {res_path}

## Validation Examples

In [ ]:
plots_path = [f for f in os.listdir(res_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

batchs_plot = [f for f in plots_path if 'val_batch' in f]
curves_plot = [f for f in plots_path if 'curve' in f]

print('Examples BBOX ')
for path in batchs_plot:
    print(''.join(path.split('/')))
    display_image(os.path.join(res_path,path), hide_axis = True)


## Classification Curves

In [ ]:
print('Curves Metrics')
for path in curves_plot:
    print(''.join(path.split('/')))
    display_image(os.path.join(res_path,path), hide_axis = True)

## Loss an MAPE

In [ ]:
display_image(os.path.join(res_path,'results.png'))

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">OCR</p>

In [ ]:
def extract_roi(image, bounding_box):
    """
    Crop the input image based on the provided bounding box coordinates.

    Args:
        image (numpy.ndarray): The input image.
        bounding_box (tuple): A tuple containing (x_min, y_min, x_max, y_max)
            coordinates of the bounding box.

    Returns:
        numpy.ndarray: The cropped image.
    """
    x_min, x_max, y_min, y_max = bounding_box
    cropped_image = image[y_min:y_max, x_min:x_max]
    return cropped_image


        

In [ ]:
os.path.join(CFG.out_folder, 'valid')+'/*.png'

In [ ]:
valid_images = glob.glob(os.path.join(CFG.out_folder, 'valid/images/*.png'))


In [ ]:
import easyocr
reader = easyocr.Reader(['en'],  gpu=True)

In [ ]:


def extract_ocr(img_path, model):
    # img = cv2.imread(img_path)
    img = Image.open(img_path).convert('RGB')
    img = np.asarray(img)

    results = model.predict(img, 
                        conf = 0.3, 
                        classes =[0], 
                        device = test_device, 
                        verbose = False)

    df_coords = pd.DataFrame(results[0].cpu().numpy().boxes.data, 
                                   columns = ['xmin', 'ymin', 'xmax', 
                                              'ymax', 'conf', 'class'])
    df_coords['class'] = df_coords['class'].replace({0:'car_plate'})

    bboxs = df_coords[['xmin','xmax','ymin','ymax']].values.astype(int)
    classes = df_coords['class'].values


    df_plate = pd.DataFrame()
    for i,bbox in enumerate(bboxs):



        roi_img = extract_roi(img, bbox)

        ocr_result = reader.readtext(np.asarray(roi_img), allowlist = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        #print(ocr_result)
        text = ocr_result[0][-2]
        #text = pytesseract.image_to_string(img)


        start_point = (bbox[0], bbox[-1]) # x_min, y_max
        end_point = (bbox[1], bbox[2])


        cv2.rectangle(img, start_point, end_point, (0,0,190), thickness=2)
        cv2.putText(img=img, text=f'{text}',
            org= (bbox[0], bbox[2]-20),
            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 0, 255),thickness=2)

    return img



In [ ]:
for path in valid_images[:5]:
    new_image = extract_ocr(path, model)
    display_image(new_image)

# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Thanks for Reading</p>

<p style="font-size:150%;text-align:center">If you liked it, leave your Upvote!</p>